### Implementing FunkSVD - Solution

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[ 10.  10.  10.  10.]
 [ 10.   4.   9.  10.]
 [  8.   9.  10.   5.]
 [  9.   8.  10.  10.]
 [ 10.   5.   9.   9.]
 [  6.   4.  10.   6.]
 [  9.   8.  10.   9.]
 [ 10.   5.   9.   8.]
 [  7.   8.  10.   8.]
 [  9.   5.   9.   7.]
 [  9.   8.  10.   8.]
 [  9.  10.  10.   9.]
 [ 10.   9.  10.   8.]
 [  5.   8.   5.   8.]
 [ 10.   8.  10.  10.]
 [  9.   9.  10.  10.]
 [  9.   8.   8.   8.]
 [ 10.   8.   1.  10.]
 [  5.   6.  10.  10.]
 [  8.   7.  10.   7.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the funtion on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of matrix factorization.

In [2]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]
    n_movies = ratings_mat.shape[1]
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat))
    
    # initialize the user and movie matrices with random values
    user_mat = np.random.rand(n_users, latent_features)
    movie_mat = np.random.rand(latent_features, n_movies)
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimizaiton Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):

        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                
                # if the rating exists
                if ratings_mat[i, j] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])
                    
                    # Keep track of the sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*diff*movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2*diff*user_mat[i, k])

        # print results for iteration
        print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))
        
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [3]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=10)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 43.541730
2 		 14.921849
3 		 4.120188
4 		 2.851237
5 		 2.719956
6 		 2.672001
7 		 2.635452
8 		 2.598694
9 		 2.558531
10 		 2.513642


In [4]:
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 10.12602327   8.73475596  10.56153387  10.24979183]
 [  8.80620304   7.39946956   8.99799041   8.79807297]
 [  7.60585098   7.1499773    8.56358846   7.52752488]
 [  9.77873588   8.06667683   9.92991945   9.67646114]
 [  8.77420543   7.20264029   8.93822472   8.56084169]
 [  6.80351173   6.06858626   7.42449309   6.66311963]
 [  9.44998425   7.8209743    9.35463661   9.46372238]
 [  8.07259988   7.34692674   8.71663697   8.08004226]
 [  8.19411121   7.51281659   9.25658395   8.30186453]
 [  7.57914069   6.47946705   8.28334871   7.69919503]
 [  9.00844943   7.93062106   9.04880367   8.941489  ]
 [  9.76668423   8.28214437   9.77175392   9.79880229]
 [  9.01063239   8.34904056   9.93862288   9.08928898]
 [  6.344798     5.96091283   7.11657586   6.53486764]
 [  9.84737329   8.43725545  10.12440054   9.83670123]
 [  9.72600573   8.51356408  10.11458998   9.85270035]
 [  8.35381086   7.3970868    8.66601671   8.20752991]
 [  7.4235392    6.07481243   7.00333846   7.3654155 ]
 [  8.3179

**The predicted ratings from the dot product are already starting to look a lot like the original data values even after only 10 iterations.  You can see some extreme low values that are not captured well yet.  The 5 in the second to last row in the first column is predicted as an 8, and the 4 in the second row and second column is predicted to be a 7.  Clearly the model is not done learning, but things are looking good.**

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [5]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 42.611941
2 		 15.164075
3 		 4.259000
4 		 2.831190
5 		 2.691763
6 		 2.655973
7 		 2.631117
8 		 2.605456
9 		 2.576642
10 		 2.543912
11 		 2.506745
12 		 2.464640
13 		 2.417098
14 		 2.363654
15 		 2.303907
16 		 2.237565
17 		 2.164492
18 		 2.084758
19 		 1.998682
20 		 1.906875
21 		 1.810259
22 		 1.710061
23 		 1.607779
24 		 1.505108
25 		 1.403837
26 		 1.305723
27 		 1.212364
28 		 1.125081
29 		 1.044843
30 		 0.972228
31 		 0.907431
32 		 0.850312
33 		 0.800468
34 		 0.757314
35 		 0.720162
36 		 0.688283
37 		 0.660964
38 		 0.637534
39 		 0.617384
40 		 0.599981
41 		 0.584858
42 		 0.571621
43 		 0.559930
44 		 0.549502
45 		 0.540096
46 		 0.531508
47 		 0.523563
48 		 0.516113
49 		 0.509029
50 		 0.502197
51 		 0.495516
52 		 0.488897
53 		 0.482256
54 		 0.475517
55 		 0.468612
56 		 0.461474
57 		 0.454041
58 		 0.446258
59 		 0.438069
60 		 0.429428
61 		 0.420289
62 		 0.410614
63 		 0.400372
64 		

In [6]:
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 10.00001871  10.00001517  10.00000574   9.99997751]
 [ 10.00001484   4.00001182   9.00000429   9.99998205]
 [  8.00001616   9.00001316  10.00000475   4.99998006]
 [  8.99999292   7.99999242   9.99999648  10.00000915]
 [  9.99999981   4.99999894   8.99999918   9.00000035]
 [  5.99999389   3.99999449   9.99999762   6.00000733]
 [  8.99996837   7.99997265   9.99998918   9.00003888]
 [  9.99999439   4.99999557   8.99999838   8.00000669]
 [  7.00002301   8.00001924  10.00000743   7.99997202]
 [  9.00001939   5.00001613   9.0000061    6.99997619]
 [  8.99996957   7.99997377   9.99998962   8.00003728]
 [  9.00002773  10.00002291  10.00000873   8.99996634]
 [  9.99999562   8.99999558   9.99999793   8.00000539]
 [  4.99996149   7.99996646   4.99998687   8.00004785]
 [  9.99993804   7.99994828   9.99998036  10.00007585]
 [  8.99998598   8.99999004   9.99999717  10.00001717]
 [  9.00001304   8.00001276   8.00000579   7.99998392]
 [ 10.00001899   8.00001662   1.00000707   9.99997745]
 [  5.0000

**In this case, we were able to completely reconstruct the item-movie matrix to obtain an essentially 0 mean squared error. I obtained 0 MSE on iteration 165.**

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [7]:
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[ nan,  10.,  10.,  10.],
        [ 10.,   4.,   9.,  10.],
        [  8.,   9.,  10.,   5.],
        [  9.,   8.,  10.,  10.],
        [ 10.,   5.,   9.,   9.],
        [  6.,   4.,  10.,   6.],
        [  9.,   8.,  10.,   9.],
        [ 10.,   5.,   9.,   8.],
        [  7.,   8.,  10.,   8.],
        [  9.,   5.,   9.,   7.],
        [  9.,   8.,  10.,   8.],
        [  9.,  10.,  10.,   9.],
        [ 10.,   9.,  10.,   8.],
        [  5.,   8.,   5.,   8.],
        [ 10.,   8.,  10.,  10.],
        [  9.,   9.,  10.,  10.],
        [  9.,   8.,   8.,   8.],
        [ 10.,   8.,   1.,  10.],
        [  5.,   6.,  10.,  10.],
        [  8.,   7.,  10.,   7.]])

In [8]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 40.389141
2 		 13.597589
3 		 4.232211
4 		 2.975619
5 		 2.782413
6 		 2.717262
7 		 2.679469
8 		 2.647932
9 		 2.616449
10 		 2.582707
11 		 2.545534
12 		 2.504131
13 		 2.457837
14 		 2.406071
15 		 2.348322
16 		 2.284167
17 		 2.213303
18 		 2.135582
19 		 2.051050
20 		 1.959990
21 		 1.862945
22 		 1.760740
23 		 1.654471
24 		 1.545481
25 		 1.435296
26 		 1.325543
27 		 1.217846
28 		 1.113711
29 		 1.014430
30 		 0.920997
31 		 0.834062
32 		 0.753931
33 		 0.680593
34 		 0.613783
35 		 0.553054
36 		 0.497859
37 		 0.447619
38 		 0.401777
39 		 0.359835
40 		 0.321374
41 		 0.286059
42 		 0.253633
43 		 0.223904
44 		 0.196727
45 		 0.171991
46 		 0.149597
47 		 0.129453
48 		 0.111457
49 		 0.095497
50 		 0.081449
51 		 0.069174
52 		 0.058527
53 		 0.049354
54 		 0.041502
55 		 0.034819
56 		 0.029161
57 		 0.024393
58 		 0.020390
59 		 0.017040
60 		 0.014244
61 		 0.011915
62 		 0.009977
63 		 0.008366
64 		

In [9]:
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 11.263048282394145:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [10]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features=4, learning_rate=0.005, iters=20)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 23.249660
2 		 10.706747
3 		 7.348244
4 		 5.677603
5 		 4.625904
6 		 3.889299
7 		 3.339789
8 		 2.913124
9 		 2.572592
10 		 2.295183
11 		 2.065631
12 		 1.873414
13 		 1.711024
14 		 1.572896
15 		 1.454759
16 		 1.353230
17 		 1.265576
18 		 1.189557
19 		 1.123320
20 		 1.065323


`6.` Now that you have a set of predictions for each user-movie pair.  Let's answer a few questions about your results. Provide the correct values to each of the variables below, and check your solutions using the tests below.

In [11]:
# How many actual ratings exist in first_1000_users
num_ratings = np.count_nonzero(~np.isnan(first_1000_users))
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()

# How many ratings did we make for user-movie pairs that didn't have ratings
ratings_for_missing = first_1000_users.shape[0]*first_1000_users.shape[1] - num_ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [12]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
